#0.初期設定

In [12]:
#*******今回の実行の設定*******

#プロジェクトのフォルダ　※BASE_DIRを、自らの環境に合わせて修正してください
BASE_DIR = '/content/drive/MyDrive/project01'

#モデル名
MODEL_NAME = "data1_trial1"

#動作モード：1:データ①、2:データ②、3:データ①コンペ時設定
ENV_MODE = 1

#乱数シード
RANDOM_SEED = 42

In [17]:
#*******Googleドライブへの接続*******
from google.colab import drive
drive.mount('/content/drive')

#*******不足ライブラリのインストール*******
!pip install sktime

#*******ライブラリのインポート*******
import sys
import os
import random
import pandas as pd
import numpy as np
import glob
import torch
from scipy.io import loadmat
%matplotlib inline

#*******乱数シードを固定*******
seed = RANDOM_SEED
torch.manual_seed(seed)  # PyTorchのシードを固定
np.random.seed(seed)     # NumPyのシードを固定
random.seed(seed)        # Pythonのrandomモジュールのシードを固定
#CUDAにおけるシードの固定
if torch.cuda.is_available():
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)  # マルチGPUを使用している場合
#CuDNNの再現性確保
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

#*******定数設定*******
DATA_DIR1 = os.path.join(BASE_DIR, 'data1/modeling_data_val')
DATA_DIR2 = os.path.join(BASE_DIR, 'data2/modeling_data_val')

#被験者
SUBJECTS=['subject0','subject1','subject2','subject3','subject4']
#クラス
CLASSES=['unknown']
PRED_CLASSES=['backside_kickturn','frontside_kickturn','pumping']

#データ属性
NUM_SENSORS = 72
SEQ_LENGTH = 250
BATCH_SIZE = 32


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#1.データローダ定義

In [7]:
import os
import math
import pandas as pd
import numpy as np
import random
import glob
import pickle
from torch.utils.data import Dataset, DataLoader

class SeqDataset(Dataset):
    def __init__(self, root, subjects, mats, mask, seq_length, is_train, transform=None, cache_file='./data_cache.pt', use_cache=True):

        self.transform = transform
        self.raw_seqs = [] #マスクで取捨選択するため、読み込んだ値を保存

        self.seqs = [] #get_itemで返す値
        self.seq_labels = []
        self.subjects = os.listdir(root)
        self.class_names = CLASSES
        self.class_names.sort()
        self.numof_classes = len(self.class_names)
        self.mask = mask
        self.seq_length = seq_length
        self.is_train = is_train
        self.cache_file = cache_file

        if os.path.exists(self.cache_file) and use_cache:
            self.load_data_from_cache()
        else:
            for (j, y) in enumerate(subjects):
                for mat in mats:
                    for (i, x) in enumerate(self.class_names):
                        temp = glob.glob(os.path.join(root, y, mat, x, '*'))
                        temp.sort()
                        self.seq_labels.extend([i] * len(temp))

                        for t in temp:
                            df = pd.read_csv(t, header=None)
                            tensor = self.preprocess(df)
                            self.raw_seqs.append(tensor)

            # ロードしたデータをキャッシュファイルに保存
            if self.cache_file!="":
                self.save_data_to_cache()

        self.seqs=self.blend_data(self.raw_seqs)

    def __getitem__(self, index):
        seq = self.seqs[index]
        if self.transform is not None:
            seq = self.transform(seq, is_train=self.is_train, seq_length=self.seq_length)

        #(タイムステップ,センサー)から(センサー,タイムステップ)に変換
        #seq = seq.T.astype(np.float32)

        return seq, self.seq_labels[index]

    def __len__(self):
        return len(self.seqs)


    #*******マスク処理関連*******
    def set_mask(self, mask):
        #マスクをセット
        self.mask = mask
        self.blend_data(self.raw_seqs)

    def blend_data(self, data1):
        #マスクに応じたデータセットマスキング処理
        seqs=[]
        for i in range(0, len(self.raw_seqs)):
            data1 = self.raw_seqs[i]

            result = np.zeros_like(data1)
            for j in range(NUM_SENSORS):
                if self.mask[j] == 1:
                    result[j, :] = data1[j, :]
                else:
                    pass #0のまま

            seqs.append(result)

        return seqs

    #*******データセットロード時の加工処理*******
    #※試行錯誤の結果、今回は標準化のみ
    def preprocess(self, df: pd.DataFrame) -> np.ndarray:
        mat = df.T.values

        #動作モード：1:データ①、2:データ②、3:データ①コンペ時設定
        if ENV_MODE!=1:  #データ1は、別途「car→標準化」を行うので標準化をパス。コンペ時はstd→car→stdを実施
            mat = self.standardization(mat, axis=1)

        return mat

    #各種前処理用関数
    def standardization(self, a, axis=None, ddof=0):
        a_mean = a.mean(axis=axis, keepdims=True)
        a_std = a.std(axis=axis, keepdims=True, ddof=ddof)
        a_std[np.where(a_std == 0)] = 1
        return (a - a_mean) / a_std

    def min_max_scaling(self, a, axis=None):
        min_val = np.min(a, axis=axis, keepdims=True)
        max_val = np.max(a, axis=axis, keepdims=True)
        return (a - min_val) / (max_val - min_val + 1e-8)

    #*******ロード高速化キャッシュ関連*******
    def save_data_to_cache(self):
        #キャッシュの保存
        os.makedirs(os.path.dirname(self.cache_file), exist_ok=True)
        with open(self.cache_file, 'wb') as f:
            pickle.dump((self.raw_seqs, self.seq_labels), f)
        print(f"キャッシュ生成：{self.cache_file}")

    def load_data_from_cache(self):
        #キャッシュの読込み
        with open(self.cache_file, 'rb') as f:
            self.raw_seqs, self.seq_labels = pickle.load(f)

#*******get_item時のデータ変換関数*******
def transform(array, is_train, seq_length):
    if is_train:
        #今回は何もしない　※Rocketにて、どれも効果が薄かったため
        ts = array[:, :seq_length].astype(np.float32)
        return ts
    else:
        ts = array[:, :seq_length].astype(np.float32)
        return ts


#2.予測実行(submit生成)

In [ ]:
import ast
import re
import torch
import numpy as np
import joblib
import os
import glob
from scipy.io import loadmat

#マスクに対応するモデル群からアンサンブルで回答を求める
#※subject(0～4) x fold(0～2) = 15モデル。各subject、foldの3モデルでアンサンブル
class ModelEnsemble:
    def __init__(self, model_dir):
        #モデルのディレクトリを指定して初期化
        self.model_dir = model_dir
        self.models = self._load_models()
        self.label_map = PRED_CLASSES

    def _load_models(self):
        #各subjectのモデルを読み込む
        models = {}
        for subject in SUBJECTS:
            subject_models = []
            for fold in range(3):  #fold0-2
                model_path = os.path.join(self.model_dir, f"{subject}_{fold}.pkl")
                if os.path.exists(model_path):
                    model = joblib.load(model_path)
                    subject_models.append(model)
            if subject_models:
                models[f"{subject}"] = subject_models
        return models

    def predict(self, subject, data):
        #指定されたsubjectのモデルでアンサンブル予測を行い、クラスラベルを返す

        if subject not in self.models:
            raise ValueError(f"モデルファイルが見つかりません：{subject}")

        # データを適切な形式に変換
        if isinstance(data, torch.Tensor):
            data = data.numpy()
        data = data.astype(np.float32)

        # 各モデルの予測を集める
        predictions = []
        for model in self.models[subject]:
            pred = model.predict(data)
            predictions.append(pred)

        # 多数決で最終予測を決定
        predictions = np.array(predictions)
        ensemble_pred = np.array([np.bincount(p).argmax() for p in predictions.T])

        # 数値から文字列ラベルに変換
        str_predictions = [self.label_map[p] for p in ensemble_pred]

        return str_predictions

#*******フィルタ適用処理*******
def apply_car_and_normalize(data):
    #Common Average Referenceと正規化を実施

    #CAR
    car_data = data - np.mean(data, axis=1, keepdims=True)

    #正規化
    mean = np.mean(car_data, axis=2, keepdims=True)
    std = np.std(car_data, axis=2, keepdims=True)
    normalized_data = (car_data - mean) / (std + 1e-8)  #1e-8：0除算回避

    return normalized_data




def predict():
    #*******予測実行*******
    case_preds_item = [[] for _ in range(3)] #各ケースでの予測結果を蓄積する [3, 問題数]
    case_preds_name = [[] for _ in range(3)]

    #動作モードによりデータ参照先変更：1:データ①、2:データ②、3:データ①コンペ時設定
    if ENV_MODE!=2:
        data_dir = DATA_DIR1
    else:
        data_dir = DATA_DIR2


    #モデルディレクトリのパスを取得
    #指定フォルダ直下のフォルダ一覧を取得
    search_dir = os.path.join(BASE_DIR, f"models_{MODEL_NAME}", "mask_models")
    case_dirs = [f for f in os.listdir(search_dir) if os.path.isdir(os.path.join(search_dir, f))]
    #フォルダ名からスコアを抽出し、降順にソート
    case_dirs = sorted(case_dirs, key=lambda x: float(re.search(r'score([0-9.]+)_', x).group(1)), reverse=True)

    #結果出力用フォルダ設定
    output_dir = os.path.join(BASE_DIR, f"models_{MODEL_NAME}", "mask_models_predictions")
    os.makedirs(output_dir, exist_ok=True)


    #上位3モデルでの予測
    for case_idx, case_dir in enumerate(case_dirs):
        model_dir = os.path.join(BASE_DIR, f"models_{MODEL_NAME}", "mask_models", case_dir)

        # モデルアンサンブルの初期化
        ensemble = ModelEnsemble(model_dir)

        # ファイルを読み込む
        with open(os.path.join(model_dir,"mask.txt"), "r") as f:
            mask_data = f.read()

        # 文字列をリストに変換
        mask = ast.literal_eval(mask_data)

        #各ケースの結果出力
        f = open(os.path.join(output_dir,f"submit_{case_dir}.csv"), "w")

        # 予測
        for subject in SUBJECTS:
            #データのロード
            val_dataset = SeqDataset(data_dir, [subject], [f'{subject}.mat'], mask=mask,
                                      seq_length=SEQ_LENGTH, is_train=False, transform=transform,
                                      cache_file='', use_cache=False)

            val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

            X_val, y_val = [], []
            for batch in val_loader:
                X, y = batch
                X_val.extend(X.numpy())
                y_val.extend(y.numpy())

            # CAR と正規化を適用
            if ENV_MODE!=2:
                #動作モードにより実施有無変更：1:データ①、2:データ②、3:データ①コンペ時設定
                #※data2はクレンジング済みなので、CARは逆効果
                X_val = apply_car_and_normalize(X_val)

            y_pred = ensemble.predict(subject, np.array(X_val))

            # リストの各項目をファイルに出力
            for i,item in enumerate(y_pred):
                idx=f"000{i}"[-3:]
                line=f"{subject}_{idx},{item}"
                f.write(f"{line}\n")

                case_preds_name[case_idx].append(f"{subject}_{idx}")
                case_preds_item[case_idx].append(item)

            print(y_pred)

        # ファイルを閉じる
        f.close()


    #*******SUBMITの出力*******
    #結果出力用
    f = open(os.path.join(output_dir,f"submit.csv"), "w")

    #多数決でsubmitを生成する
    for y in range(len(case_preds_name[0])): #問題数分、ループ
        a=case_preds_item[0][y]
        b=case_preds_item[1][y]
        c=case_preds_item[2][y]
        if a==b or a==c:
            item = a
        elif b==c:
            item = b
        else:
            item = a

        line=f"{case_preds_name[0][y]},{item}"
        f.write(f"{line}\n")

    # ファイルを閉じる
    f.close()

predict()